In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import os
from langame.langame_client import LangameClient
import random
import openai
from firebase_admin import firestore
import glob

c = LangameClient()

In [2]:
memes = []
for e in c._firestore_client.collection('memes').stream():
    memes.append((e.id, e.to_dict()))

In [3]:
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = [model.encode(e[1]['content']) for e in memes]

In [6]:
np.save("embeddings/part1.npy", embeddings)

In [ ]:
from autofaiss.external.quantize import Quantizer
quantizer = Quantizer()
quantizer.quantize(embeddings_path="embeddings", output_path="indexes", max_index_memory_usage="6G", current_memory_available="8G")

In [5]:
from glob import glob
# get all files in ../data/augment/*.txt
files = glob("../data/augment/*.txt")
# make a single string out of it
text = "\n".join([open(f, 'r').read() for f in files]).split("\n")
len(text)

56200

In [6]:
import re
"""
[' ("How does the universe expand and how do you know it doesn\'t expand forever", [\'physic\']),',
 " ('Is there a difference between self-reference and self-observation?', ['psychology']),",
 " ('What were you doing before you got into philosophy ?', ['knowledge']),",
 " ('Is it true that a smarter AI will have more rights than a human being?', ['artificial intelligence']),",
"""
# regex that take only alphanumeric characters
regex = re.compile(r'[^a-zA-Z0-9]')
nd = []
for e in text:
    parsed = [e.strip() for e in regex.sub(' ', e).split('     ')]
    nd.append(parsed[0])
len(nd)

56200

In [28]:
import faiss
import glob
from sentence_transformers import SentenceTransformer

my_index = faiss.read_index(glob.glob("indexes/*.index")[0])
model = SentenceTransformer('all-mpnet-base-v2')

# print the results with their distance
for e in nd[:10]:
    k = 5
    xq = model.encode(e)
    print(f"sentence: '{e}'")
    distances, indices = my_index.search(np.array([xq]), k)
    for i in range(k):
        score = distances[0][i]
        if score > 0.7:
            print(f"found with distance {score} sentence: {nd[indices[0][i]]}")


sentence: 'How does the universe expand and how do you know it doesn t expand forever'
sentence: 'Is there a difference between self reference and self observation'
sentence: 'What were you doing before you got into philosophy'
sentence: 'Is it true that a smarter AI will have more rights than a human being'
sentence: 'Is it possible to create a machine that has consciousness'
found with distance 0.7920733690261841 sentence: Mathematics draws on a host of general Western and Eastern philosophies
found with distance 0.7916989922523499 sentence: In case mathematic become nanotechnology is that possible    what kind of data inside the bits will be most advanced  binary  or BETA    n
found with distance 0.7721134424209595 sentence: Will human beings ever be able to fully understand everything that  s around us
found with distance 0.7292589545249939 sentence: Could you give me a formal proof to show that  3 2 2x3
sentence: 'What is your opinion about scientists in general'
found with distan

In [27]:
from langame.strings import group_by_similarity_distinct, group_by_similarity
list(group_by_similarity(nd[:1000]))

[['How does the universe expand and how do you know it doesn t expand forever'],
 ['Is there a difference between self reference and self observation'],
 ['What were you doing before you got into philosophy'],
 ['Is it true that a smarter AI will have more rights than a human being'],
 ['Is it possible to create a machine that has consciousness'],
 ['What is your opinion about scientists in general'],
 ['What is the purpose of human life',
  'What is the purpose of human life',
  'What is the purpose of IQ',
  'What is the purpose of mind',
  'What is the purpose of killing',
  'What is the purpose of life',
  'What is the purpose of human life  n'],
 ['How do you feel about robots becoming part of society in the future',
  'How do you feel about robots becoming part of society in the future'],
 ['Will AI take over the world  n'],
 ['Will technology inevitably lead to the creation of a superintelligence'],
 ['When did you first realize that you wanted to be a computer scientist'],
 ['W